##### Copyright 2025 Google LLC.

In [24]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [25]:
%pip install -q -U "google-genai>=1.0.0"

In [26]:
%pip install -q wikipedia

Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [27]:
import json
import textwrap

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [28]:
from google.colab import userdata
from google import genai
API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

## Define tools

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Step 1: Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [34]:
from google.genai import types
from typing import List

# This is the actual function that would be called based on the model's suggestion
# Define the function with type hints and docstring
def wikipedia_search(search_queries: List[str]) -> List[str]:
  """Search wikipedia for each query and summarize relevant docs.
    Args:
        search_queries: The user query to search wikipedia.

    Returns:
        A list of relevant information from wikipedia based on the query.
  """
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []

  # Note: newer models have more limited quota for API calls
  MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
  model=f"models/{MODEL_ID}"
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = client.models.generate_content(model=model,
                                                  contents=textwrap.dedent(f"""\
                                                        Extract relevant information
                                                        about user's query: {query}
                                                        From this source:

                                                        {page}

                                                        Note: Do not summarize. Only Extract and return the relevant information
                                                        """),)
        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [30]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Retrieval-augmented generation', 'Vibe coding']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Large language model did not match with any page id, hence skipping.
Fetching page: "Retrieval-augmented generation"
Information Source: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
Retrieval-augmented generation did not match with any page id, hence skipping.
Fetching page: "Vibe coding"
Information Source: https://en.wikipedia.org/wiki/Vibe_coding
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Retrieval-augmented_generation
     https://en.wikipedia.org/wiki/Vibe_coding


Here is what the search results look like:

In [31]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> *   LLMs are large language models tuned for coding, used in vibe coding to generate software from natural language prompts.
> *   Vibe coding relies on LLMs, allowing programmers to generate working code by providing natural language descriptions rather than manually writing it.
> *   The capabilities of LLMs were such that humans would no longer need to learn specific programming languages to command computers.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Vibe_coding

### Step 2: Automatic Function Calling (Python Only)

When using the Python SDK, you can provide Python functions directly as tools. The SDK automatically converts the Python function to declarations, handles the function call execution and response cycle for you. The Python SDK then automatically:

- Detects function call responses from the model.
- Call the corresponding Python function in your code.
- Sends the function response back to the model.
- Returns the model's final text response.

To use this, define your function with type hints and a docstring, and then pass the function itself (not a JSON declaration) as a tool:

Note: This approach only handles annotations of `AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])`

In [35]:
config = types.GenerateContentConfig(
    tools=[wikipedia_search]
) # Pass the function itself

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [33]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

Now start a new chat with `enable_automatic_function_calling=True`. With it enabled, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [36]:
MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
model=f"models/{MODEL_ID}"

chat = client.chats.create(model=model, config=config, history=[])

query = "Explain how deep-sea life survives."

response = chat.send_message(instructions.format(query=query))

Searching for "deep-sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Deep sea did not match with any page id, hence skipping.
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Deep-sea fish did not match with any page id, hence skipping.
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Deep-sea gigantism did not match with any page id, hence skipping.
Searching for "deep-sea ecosystems"
Related search terms: ['Deep-sea community', 'Deep sea mining', 'Deep sea']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Deep sea mining did not match with any page id, hence skipping.
Searching for "hydrothermal vent 

In [37]:
to_markdown(response.text)

> Deep-sea life survives in a challenging environment characterized by perpetual darkness, immense pressure, cold temperatures, and limited food sources. Here's a breakdown of the key adaptations and strategies:
> 
> *   **Energy Sources:** Deep-sea communities rely on three primary energy sources: marine snow (particulate organic matter sinking from the photic zone), whale falls (carcasses providing a large input of organic matter), and chemosynthesis at hydrothermal vents and cold seeps. Chemosynthesis is a process where bacteria use chemicals like hydrogen sulfide and methane to produce energy, forming the base of the food web in these unique environments.
> *   **Adaptations to Pressure:** Organisms have developed several adaptations to cope with extreme pressure, including small size, gelatinous flesh, minimal skeletal structure, and the elimination of excess cavities.
> *   **Zones:** The deep sea is divided into zones: mesopelagic, bathyal, abyssal, and hadal, each with distinct characteristics and inhabitants adapted to specific conditions.
> *   **Chemosynthesis:** At hydrothermal vents and cold seeps, chemosynthetic bacteria provide energy for the ecosystem.
> *   **Food webs:** Deep-sea food webs are complex and still being studied. Predator-prey interactions, the role of gelatinous zooplankton, and the impact of deep-sea mining are important areas of research.
> *   **Darkness:** Adaptations include reliance on material sinking from above or chemosynthesis.


Check for additional citations:

In [39]:
response.candidates[0].citation_metadata or 'No citations found'

'No citations found'

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [43]:
for content in chat.get_history():
    display(Markdown("###" + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print("Function call: {", part.function_call, "}")
        if part.function_response:
            print("Function response: {", part.function_response, "}")
    print("-" * 80)


###user:

You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: Explain how deep-sea life survives.


--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'search_queries': ['deep-sea life adaptations', 'deep-sea ecosystems', 'hydrothermal vent life', 'deep-sea food sources', 'pressure effects on deep-sea life', 'deep-sea bioluminescence', 'challenges of deep-sea environment', 'how do organisms survive in the deep sea?', 'deep sea chemosynthesis']} name='wikipedia_search' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='wikipedia_search' response={'result': ['*   **Definition:** A deep-sea community is any community of organisms associated by a shared habitat in the deep sea.\n*   **Challenges:** Deep-sea communities are largely unexplored due to technological, logistical challenges, and expense.\n*   **Historical Beliefs:** It was once believed that little life existed in the deep sea due to the harsh conditions.\n*   **Biodiversity:** Research has demonstrated significant biodiversity in the deep sea.\n*   **Energy Sources:** The three main sources of energy and nutrients for deep-sea communities are:\n    *   Marine snow\n    *   Whale falls\n    *   Chemosynthesis at hydrothermal vents and cold seeps\n*   **Discovery of Chemosynthesis:** The first deep-sea chemosynthetic community was discovered at hydrothermal vents in the eastern Pacific Ocean in 1977.\n*   **Challenger Deep:** The deepest surveyed point in Earth\'s oceans, located in the Mariana Trench.\n*   *

###model:

Deep-sea life survives in a challenging environment characterized by perpetual darkness, immense pressure, cold temperatures, and limited food sources. Here's a breakdown of the key adaptations and strategies:

*   **Energy Sources:** Deep-sea communities rely on three primary energy sources: marine snow (particulate organic matter sinking from the photic zone), whale falls (carcasses providing a large input of organic matter), and chemosynthesis at hydrothermal vents and cold seeps. Chemosynthesis is a process where bacteria use chemicals like hydrogen sulfide and methane to produce energy, forming the base of the food web in these unique environments.
*   **Adaptations to Pressure:** Organisms have developed several adaptations to cope with extreme pressure, including small size, gelatinous flesh, minimal skeletal structure, and the elimination of excess cavities.
*   **Zones:** The deep sea is divided into zones: mesopelagic, bathyal, abyssal, and hadal, each with distinct characteristics and inhabitants adapted to specific conditions.
*   **Chemosynthesis:** At hydrothermal vents and cold seeps, chemosynthetic bacteria provide energy for the ecosystem.
*   **Food webs:** Deep-sea food webs are complex and still being studied. Predator-prey interactions, the role of gelatinous zooplankton, and the impact of deep-sea mining are important areas of research.
*   **Darkness:** Adaptations include reliance on material sinking from above or chemosynthesis.


--------------------------------------------------------------------------------


In the chat history you can see all 4 steps:

1. User: Asks the question about the total number of mittens.
2. Model: Determines that the `wikipedia_search` is helpful and sends a FunctionCall request to the user.
3. User: The Chat session automatically executes the function (due to `_automatic_function_calling` is enabled by default) and sends back a FunctionResponse with the calculated result.
4. Model: Uses the function's output to formulate the final answer and presents it as a text response.

## [Optional] Manually execute the function call

If you want to understand what happened behind the scenes, this section executes the `genai.types.FunctionCall` manually to demonstrate.

In [98]:
config  = {
        "tools": [wikipedia_search],
        "automatic_function_calling": {"disable": True},
    }

chat    = client.chats.create(model=model, config=config, history=[])
response  = chat.send_message(instructions.format(query=query))

Initially the model returns a FunctionCall:

In [99]:
fc = response.candidates[0].content.parts[0].function_call
print(fc)
print(f"FunctionCall's name is {fc.name}")

id=None args={'search_queries': ['Deep-sea life adaptations', 'Deep-sea ecosystems', 'How do deep-sea organisms get energy?', 'Deep-sea food web', 'What are hydrothermal vents?', 'What is chemosynthesis?', 'Deep-sea pressure adaptations', 'Deep-sea temperature adaptations', 'Deep-sea animal adaptations', 'Bioluminescence in deep-sea creatures', 'Deep-sea survival strategies']} name='wikipedia_search'
FunctionCall's name is wikipedia_search


Call the function with generated arguments to get the results.

In [100]:
if fc.name == "wikipedia_search":
    summaries = wikipedia_search(**fc.args)
    print(f"Function execution result: {summaries}")

Searching for "Deep-sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "Deep-sea ecosystems"
Related search terms: ['Deep-sea community', 'Deep sea mining', 'Deep sea']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Searching for "How do deep-sea organisms get energy?"
Related search terms: ['Deep sea mining', 'Hydrothermal vent', 'Sea']
Fetching page: "Hydrothermal vent"
Information Source: https://en.wikipedia.org/wiki/Hydrothermal_vent
Hydrothermal vent did not match with any page id, hence 

Now send the `summaries` to the model.

In [103]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=fc.name,
    response={"result": summaries},
)

response = chat.send_message(function_response_part)

to_markdown(response.text)

> Deep-sea life survives in a challenging environment characterized by darkness, high pressure, and low temperatures. Here's a breakdown of their survival strategies:
> 
> *   **Energy Sources:** Since sunlight doesn't penetrate the deep sea, organisms rely on alternative energy sources like marine snow (organic matter sinking from upper waters), whale falls (carcasses), and chemosynthesis (energy from chemicals at hydrothermal vents and cold seeps).
> *   **Adaptations to Pressure:** Deep-sea creatures have developed adaptations at the protein, anatomical, and metabolic levels to withstand immense hydrostatic pressure. Some have altered protein mechanisms and accumulate specific osmolytes like Trimethylamine N-oxide (TMAO) to protect proteins.
> *   **Temperature Adaptations:**  Deep-sea gigantism, where animals grow larger than their shallow-water relatives, is linked to colder temperatures, food scarcity, and reduced predation.
> *   **Sensory Adaptations:** Many deep-sea fish have large, sensitive eyes to detect bioluminescence or are blind.
> *   **Bioluminescence:**  Many species use bioluminescence to attract prey, find mates, distract predators, and camouflage.
> *   **Buoyancy:** Organisms often have jelly-like flesh, minimal bone structure, and high fat content to maintain buoyancy.
> *   **Feeding Adaptations:**  Deep-sea fish possess adaptations for consuming large prey, such as sharp teeth, hinged jaws, large mouths, and expandable bodies.
> *   **Hydrothermal Vents:** Some species near hydrothermal vents rely on chemosynthesis, where bacteria convert chemicals into energy. Symbiotic relationships, like those between tube worms and chemosynthetic bacteria, are common.


## Re-ranking the search results

Helper function to embed the content:

In [104]:
# def get_embeddings(content: list[str]) -> np.ndarray:
#   embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
#   embds = embeddings.get('embedding', None)
#   embds = np.array(embds).reshape(len(embds), -1)
#   return embds
from tqdm.auto import tqdm
from google.genai import types

tqdm.pandas()

from google.api_core import retry
import numpy as np

def make_embed_text_fn(model):

    @retry.Retry(timeout=300.0)
    def embed_fn(texts: list[str]) -> list[list[float]]:
        # Set the task_type to CLASSIFICATION and embed the batch of texts
        embeddings = client.models.embed_content(
            model=model,
            contents=texts,
            config=types.EmbedContentConfig(task_type="CLASSIFICATION"),
        ).embeddings
        return np.array([embedding.values for embedding in embeddings])

    return embed_fn


def create_embeddings(content: list[str]) -> np.ndarray:
    MODEL_ID = "text-embedding-004" # @param ["embedding-001", "text-embedding-004","gemini-embedding-exp-03-07"] {"allow-input":true, isTemplate: true}
    model = f"models/{MODEL_ID}"
    embed_fn = make_embed_text_fn(model)

    batch_size = 100  # at most 100 requests can be in one batch
    all_embeddings = []

    # Loop over the texts in chunks of batch_size
    for i in tqdm(range(0, len(content), batch_size)):
        batch = content[i:i + batch_size]
        embeddings = embed_fn(batch)
        all_embeddings.extend(embeddings)

    return np.array(all_embeddings).reshape(len(all_embeddings), -1)

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [105]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [106]:
search_res = create_embeddings(summaries)
embedded_query = create_embeddings([query])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity score:

In [107]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [109]:
to_markdown(summaries[np.argmax(sim_value)])

> The deep sea is defined as the ocean depth where light begins to fade, approximately 200 m (660 ft). Conditions include low temperatures, darkness, and high pressure.
> 
> Organisms in the deep sea have various adaptations to survive, utilizing feeding methods like scavenging, predation, and filtration. Many feed on marine snow, which is organic material falling from upper waters.
> 
> **Environmental characteristics:**
> 
> *   **Light:** Natural light does not penetrate, except in the upper mesopelagic. No photosynthesis occurs, so life depends on energy from elsewhere, mainly marine snow (algal particulates, detritus, and biological waste).
> *   **Pressure:** Pressure increases by about 1 atmosphere per 10 meters of depth, resulting in extreme pressure.
> *   **Salinity:** Relatively constant at about 35 parts per thousand.
> *   **Temperature:** Greatest temperature gradient at the thermocline and hydrothermal vents. Below the thermocline, the water is cold and homogeneous.
> 
> **Biology:**
> 
> *   Regions are divided into bathyal (200-3,000 m), abyssal (3-6 km), and hadal (6-11 km) zones. Food is scarce and consists of marine snow and carcasses.
> *   Many species have jelly-like flesh for buoyancy instead of relying on gas.
> *   Midwater fish are small with slow metabolisms, weak muscles, and often have extendable jaws. Hermaphroditism is common due to difficulty finding partners.
> *   Fish often have large, tubular eyes with only rod cells for upward vision. Prey fish have adaptations like lateral compression and counter illumination (bioluminescence) to reduce silhouettes. Some fish have retroreflectors behind the retina.
> *   Organisms rely on sinking organic matter, with only 1-3% of surface production reaching the seabed as marine snow. Larger food falls, like whale carcasses, also occur. Filter feeders consume organic particles.
> *   Marine bacteriophages cycle nutrients in deep-sea sediments.
> 
> **Chemosynthesis:**
> 
> *   Some species at hydrothermal vents rely on chemosynthesis, like the symbiotic relationship between tube worms and chemosynthetic bacteria, not sunlight.
> 
> **Adaptation to hydrostatic pressure:**
> 
> *   Deep sea fish have adaptations in their proteins, anatomical structures, and metabolic systems to withstand great amount of hydrostatic pressure.
> *   Hydrostatic pressure affects both protein folding and assembly and enzymatic activity
> *   Some Deep-sea fish developed pressure tolerance through the change in mechanism of their α-actin.
> *   Specific osmolytes like Trimethylamine N-oxide (TMAO) are abundant in deep sea fish and protect proteins from high hydrostatic pressure.
> *   Mariana hadal snailfish have a modification in the Osteocalcin gene, leading to an open skull and cartilage-based bone formation.
> 
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep_sea

### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-2.0-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [111]:
res = client.models.generate_content(model=model,
                                contents=textwrap.dedent(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}"""),)

to_markdown(res.text)

> Okay, so imagine the deep sea, right? Forget everything you think you know about a nice, sunny beach. We're talking about a world of permanent night, crushing pressure, and... well, let's just say the menu isn't exactly five-star. So how does anything actually *live* down there? It's a pretty wild story, actually, relying on a combination of adaptations and opportunistic resourcefulness.
> 
> First off, forget photosynthesis. No sunlight, no plants. So, everything boils down to what we call "marine snow." Think of it like a ghostly blizzard of organic particles – dead plankton, fecal matter from creatures above, bits of decaying stuff in general – slowly sinking down from the sunlit zone. This is the primary food source for many deep-sea creatures. Some, like certain [Placeholder for Deep-Sea Worm Example], are specialized filter feeders, patiently waiting to catch the falling snow with their [Placeholder for Appendage Description].
> 
> But that marine snow isn't enough for everyone. That's where the hunters come in. Many deep-sea creatures are predators, and they've evolved some truly remarkable (and often terrifying!) adaptations. Think about the [Placeholder for Anglerfish Example] with its bioluminescent lure. It's like a tiny, glowing dinner bell ringing in the eternal darkness, attracting unsuspecting prey. Or consider the [Placeholder for Viperfish Example] with its huge jaws and dagger-like teeth, perfectly adapted for snatching up anything that comes close.
> 
> Speaking of bioluminescence, it's HUGE down there. It's not just for attracting prey. Creatures use it for all sorts of things: communication, camouflage, even defense. For example, the [Placeholder for Deep Sea Squid Example] might flash a dazzling light to startle a predator, giving it a chance to escape.
> 
> And then there's the pressure. We're talking about immense forces, enough to crush a submarine. But deep-sea creatures have adapted to that too. Their bodies often lack air-filled cavities, which would collapse under pressure. Instead, they're filled with [Placeholder for Fluid or Tissue Type] that helps equalize the pressure inside and outside their bodies.
> 
> Finally, you can't forget about hydrothermal vents. These are like underwater geysers, spewing out hot, chemical-rich water from deep within the Earth. They're the basis for entirely different ecosystems, where bacteria thrive on the chemicals and form the base of a food web that supports all sorts of specialized creatures like [Placeholder for Vent Worm Example] and [Placeholder for Vent Crab Example].
> 
> So, surviving in the deep sea is all about adapting to extreme conditions, making the most of limited resources, and, let's be honest, being a little bit weird and wonderful. It's a challenging place to live, but life, as always, finds a way! And we're still discovering new and amazing deep-sea creatures all the time. Who knows what other secrets are hidden in the depths?


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [112]:
hypothetical_ans = create_embeddings([res.text])

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity scores to rank the search results

In [113]:
sim_value = dot_product(search_res, hypothetical_ans)
sim_value

array([[0.92962056],
       [0.91904977],
       [0.91857284],
       [0.92877319],
       [0.90089099],
       [0.87556743],
       [0.75732358],
       [0.82864696]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [114]:
to_markdown(result[np.argmax(sim_value)])

> The document contains information relevant to deep-sea adaptations for survival in the following ways:
> 
> *   **Challenges of the Deep Sea:** High pressure, extreme temperatures, and absence of light pose significant challenges.
> *   **Darkness (Lack of Light):**
>     *   The absence of sunlight in the aphotic zone necessitates alternative energy sources or movement to the photic zone.
>     *   Mesopelagic organisms have large eyes and use bioluminescence for camouflage, communication, and predation.
>     *   Red or black coloration functions as camouflage in the absence of red wavelengths of light.
> *   **Hyperbaricity (High Pressure):**
>     *   Deep-sea animals have evolved to withstand extreme pressure, with adaptations such as:
>         *   Small body size (usually not exceeding 25cm).
>         *   Gelatinous flesh and minimal skeletal structure.
>         *   Elimination of excess cavities like swim bladders.
> *   **Temperature:**
>     *   Deep-sea temperatures are generally cold and constant.
>     *   Hydrothermal vents are an exception, with extreme temperature gradients.
> *   **Energy Sources:**
>     *   **Marine Snow:** Reliance on sinking organic matter from the photic zone.
>     *   **Occasional Surface Blooms:** Utilization of plankton blooms.
>     *   **Whale Falls:** Consumption of the organic matter in whale carcasses.
>     *   **Chemosynthesis:**
>         *   Chemosynthetic bacteria at hydrothermal vents and cold seeps provide energy for the food web.
> *   **Adaptations in Different Zones:**
>     *   **Mesopelagic:** Vertical migrations, large eyes, bioluminescence, coloration (red, black, silvery).
>     *   **Bathyal:** Reduced organs (gills, kidneys, hearts, swimbladders), weak skeletal and muscular build, slimy skin.
>     *   **Abyssal/Hadal:** Tolerance to immense pressure, chemotrophic lifestyles.
> *   **Deep sea environment's parameters:**
> 
>     *   Depth: below the thermocline, at a depth of 1,000 fathoms (1,800 m) or more.
>     *   Pressure: between 200 and 600 atm, the range of pressure is from 20 to 1,000 atm.
>     *   Temperature: water is cold and far more homogeneous, from 5 or 6 °C at 1,000 meters, and isothermal below 3,000 to 4,000 m.
>     *   Salinity: constant throughout the depths.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep-sea_community

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.